In [ ]:
import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/auto-diagnosis/')
%cd /home/schirrmr/
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'

%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png' 
matplotlib.rcParams['font.size'] = 7

import matplotlib.lines as mlines
import seaborn
seaborn.set_style('darkgrid')
matplotlib.rcParams['figure.figsize'] = (12.0, 3.0)
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('DEBUG')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)
from numpy.random import RandomState
from braindecode.torch_ext.util import np_to_var
from autodiag.perturbation import combine_covs, combine_vars

In [ ]:
crop_preds = np.load('data/models/pytorch/auto-diag/6000-input/24/test_crop_preds.npy')
trial_labels = np.load('data/models/pytorch/auto-diag/6000-input/24/test_trial_labels.npy')

In [ ]:
crop_preds[0].shape

In [ ]:
np.concatenate(crop_preds[0], axis=1).shape

In [ ]:
trial_preds = [np.concatenate(c[:-1], axis=1) for c in crop_preds]

In [ ]:
np.mean(np.argmax([np.mean(c, axis=(0,2)) for c in crop_preds], axis=1) == trial_labels)

In [ ]:
max_shape = np.max([list(t.shape) for t in trial_preds], axis=0)

In [ ]:
max_shape

In [ ]:
preds_padded = np.full((len(trial_preds), max_shape[1], max_shape[0]),np.nan)

for i_trial, trial_pred in enumerate(trial_preds):
    preds_padded[i_trial,:trial_pred.shape[1]] =  trial_pred.T

nan_mask = (0 * preds_padded + 1)[:,:,0]

labels_padded = np.argmax(preds_padded, axis=2)

pred_correct = labels_padded == trial_labels[:,None]

pred_correct = pred_correct * nan_mask

correct_timecourse = np.nanmean(pred_correct, axis=0) * 100
correct_std_timecourse = np.nanstd(pred_correct, axis=0) * 100

In [ ]:
plt.plot((600 + np.arange(len(correct_timecourse))) / 100.0,correct_timecourse)
plt.fill_between((600 + np.arange(len(correct_timecourse))) / 100.0,
                 correct_timecourse - correct_std_timecourse / np.sqrt(len(pred_correct)),
                 correct_timecourse + correct_std_timecourse / np.sqrt(len(pred_correct)),
            alpha=0.3)
plt.xlabel('Time [sec]', fontsize=14)
plt.ylabel('Accuracy [%]', fontsize=14)

In [ ]:
plt.plot((600 + np.arange(len(correct_timecourse))) / 100.0,correct_timecourse)
plt.fill_between((600 + np.arange(len(correct_timecourse))) / 100.0,
                 correct_timecourse - correct_std_timecourse / np.sqrt(len(pred_correct)),
                 correct_timecourse + correct_std_timecourse / np.sqrt(len(pred_correct)),
            alpha=0.5)
plt.xlabel('Time [sec]', fontsize=14)
plt.ylabel('Accuracy [%]', fontsize=14)

In [ ]:
from rtsutil.stats import running_mean

In [ ]:
n_smooth_samples = 1200
smoothed_timecourse = running_mean(correct_timecourse,n_smooth_samples,)
smoothed_std = running_mean(correct_std_timecourse,n_smooth_samples,)
plt.plot((n_smooth_samples / 2 + 600 + np.arange(len(smoothed_timecourse))) / 100.0, smoothed_timecourse)


plt.fill_between((n_smooth_samples / 2 + 600 + np.arange(len(smoothed_timecourse))) / 100.0,
                 smoothed_timecourse - smoothed_std / np.sqrt(len(pred_correct)),
                 smoothed_timecourse + smoothed_std / np.sqrt(len(pred_correct)),
            alpha=0.5)
plt.xlabel('Time [sec]', fontsize=14)
plt.ylabel('Accuracy [%]', fontsize=14)
plt.title("2-second smoothing", fontsize=16)

In [ ]:
n_smooth_samples = 3000
smoothed_timecourse = running_mean(correct_timecourse,n_smooth_samples,)
smoothed_std = running_mean(correct_std_timecourse,n_smooth_samples,)
plt.plot((n_smooth_samples / 2 + 600 + np.arange(len(smoothed_timecourse))) / 100.0, smoothed_timecourse)


plt.fill_between((n_smooth_samples / 2 + 600 + np.arange(len(smoothed_timecourse))) / 100.0,
                 smoothed_timecourse - smoothed_std / np.sqrt(len(pred_correct)),
                 smoothed_timecourse + smoothed_std / np.sqrt(len(pred_correct)),
            alpha=0.5)
plt.xlabel('Time [sec]', fontsize=14)
plt.ylabel('Accuracy [%]', fontsize=14)
plt.title("5-second smoothing", fontsize=16)

In [ ]:
n_smooth_samples = 60000
smoothed_timecourse = running_mean(correct_timecourse,n_smooth_samples,)
smoothed_std = running_mean(correct_std_timecourse,n_smooth_samples,)
plt.plot((n_smooth_samples / 2 + 600 + np.arange(len(smoothed_timecourse))) / 100.0, smoothed_timecourse)


plt.fill_between((n_smooth_samples / 2 + 600 + np.arange(len(smoothed_timecourse))) / 100.0,
                 smoothed_timecourse - smoothed_std / np.sqrt(len(pred_correct)),
                 smoothed_timecourse + smoothed_std / np.sqrt(len(pred_correct)),
            alpha=0.5)
plt.xlabel('Time [sec]', fontsize=14)
plt.ylabel('Accuracy [%]', fontsize=14)
plt.title("5-second smoothing", fontsize=16)